In [1]:
# Scrape LinkedIn companies based on user inputted search query and update into Company Web Scrape Google Sheets 
# Script for LinkedIn Company Mining Feature

#import libraries
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import time, sys, requests, random

# import and authorize gspread  
import gspread
from oauth2client.service_account import ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
google_key_file = '/Users/zacharywong/Documents/ServiceAccountKey-Secret/pelagic-tracker-338302-eaf0e0e671cb.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(google_key_file, scope)
gc = gspread.authorize(credentials)

In [2]:
#Global variables
query = ''
numberofPages = 0
cellLocationQuery = 'C2'
cellLocationPages = 'B2'
sheetIndexRead = 0
sheetIndexWrite = 2
columnName = 'Company Names (Leave Blank if Using LinkedIn Company Mining)'
password = ''
email = 'zach.jl.wong@gmail.com'

#paths/urls
spreadsheet_id = '1vFXonFCyUlEKa1f0s5tvHCKeTek_sAv7rUPYfYss0Qo'
loginurl = 'https://www.linkedin.com/uas/login'
driver_path = '/Users/zacharywong/Documents/Work/Portfolio/DigitalHealthWebscrape/chromedriver'
pathtoLinkedInFile = '/Users/zacharywong/github/zacharywong2023/DigitalHealthWebscrape/Misc/Companies/Company Web Scrape Tool - Company Names.csv'
pathtoPassword = '/Users/zacharywong/Documents/LinkedIn/LinkedIn-Password-Secret.txt'

#waitTimes
waitLinkedInTime = 10
waitCaptcha = 25
waitSearch= 1
waitLogin = 0.8


# xPaths, classes, and javascripts
searchBarXPath = '//*[@id="global-nav-typeahead"]/input'
companyFilterXPath = '//*[@id="search-reusables__filters-bar"]/ul/li[2]/button'
scrollDownScript = "window.scrollTo(0,document.body.scrollHeight)"
pageButtonClass = "button[type='button']"
companyTitleClass = 'app-aware-link'
companyTitleAttr = 'a'


In [3]:
# helper function to read in value from spreadsheet
def readinValue(cellLocation, sheetIndexRead):
    sh = gc.open_by_key(spreadsheet_id)
    worksheet = sh.get_worksheet(sheetIndexRead)
    try:
        value = worksheet.acell(cellLocation).value
    except:
        wait()
    return value

In [4]:
# read in query from spreadsheet
def readinQuery():
    global query
    query = readinValue(cellLocationQuery, sheetIndexRead)
    print('query: ' + query)

In [5]:
# read in number of pages from spreadsheet
def readinPages():
    global numberofPages
    numberofPages = int(readinValue(cellLocationPages, sheetIndexRead))
    print('numberofPages: ', numberofPages)

In [6]:
# login to linkedin 
def login(driver):
    driver.get(loginurl)
    emailInput = driver.find_element(By.ID, 'username')
    emailInput.send_keys(email)
    time.sleep(waitLogin)
    passwordInput = driver.find_element(By.ID, 'password')
    with open (pathtoPassword, 'r') as file:
        global password
        password = file.read()
    passwordInput.send_keys(password)
    time.sleep(waitLogin)
    passwordInput.send_keys(Keys.RETURN)
    

In [7]:
# search query on LinkedIn
def searchLinkedIn(driver):
    #search query
    try:
        searchBar = driver.find_element(By.XPATH, searchBarXPath)
        searchBar.send_keys(query)
        searchBar.send_keys(Keys.RETURN)
        time.sleep(1)
    except: 
        time.sleep(waitCaptcha)
        searchBar = driver.find_element(By.XPATH, searchBarXPath)
        searchBar.send_keys(query)
        searchBar.send_keys(Keys.RETURN)
        time.sleep(1)

In [8]:
# Click Company Filter option on LinkedIn
def companyFilter(driver):
    WebDriverWait(driver, waitLinkedInTime).until(EC.presence_of_element_located((By.XPATH, companyFilterXPath)))
    companyButtons = driver.find_element(By.XPATH, companyFilterXPath)
    companyButtons.click()

In [9]:
# Click onto next page 
def nextPage(currentPage, driver):
    driver.execute_script(scrollDownScript)
    time.sleep(1)
    #WebDriverWait(driver, waitLinkedInTime).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ember329"]/button')))
    pages = driver.find_elements_by_css_selector(pageButtonClass)
    for page in pages:
        #time.sleep(1)
        buttonNumber = page.text
        #print(buttonNumber)
        if buttonNumber == str(currentPage):
            #print('hello')
            page.click()
            break

In [10]:
# add companies into a pandas dataframe 
def addCompanies(driver):
    companies = []
    currentPage = 1
    while (currentPage <= numberofPages):
        time.sleep(2)
        soup = bs(driver.page_source, 'html.parser')
        companySearch = soup.find_all(companyTitleAttr, class_ = companyTitleClass)
        for company in companySearch:
            companyName = company.text
            if '\n\n\n' not in companyName and companyName not in companies:
                companies.append(companyName.replace('\n', ''))
        currentPage += 1
        nextPage(currentPage, driver)
    return companies

In [11]:
# export final list of companies to CSV 
def exportCSV(companies):
    df = pd.DataFrame(companies, columns = [columnName])
    df.to_csv(pathtoLinkedInFile)
    return df

In [12]:
# Update spreadsheet with list of companies
def updateSpreadSheet(df, sheetIndexWrite):
    sh = gc.open_by_key(spreadsheet_id)
    worksheet = sh.get_worksheet(sheetIndexWrite)
    worksheet.clear()
    worksheet.update([df.columns.values.tolist()] + df.values.tolist())

In [13]:
# run all functions to scrape LinkedIn companies  
def runLinkedIn():
    service = Service(driver_path)
    driver = webdriver.Chrome(service = service)
    readinQuery()
    readinPages()
    login(driver)
    searchLinkedIn(driver)
    companyFilter(driver)
    companies = addCompanies(driver)
    df = exportCSV(companies)
    updateSpreadSheet(df, sheetIndexWrite)
    return df

In [14]:
# run program 
startTime = time.time()
df = runLinkedIn()
endTime = time.time()
elapsedTime = endTime - startTime
print('Time Elapsed: ', elapsedTime)
df

query: Digital Health
numberofPages:  3


/var/folders/gl/jgk7f2w528qgjggck6glqp200000gn/T/ipykernel_7760/2188253913.py:6: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  pages = driver.find_elements_by_css_selector(pageButtonClass)


Time Elapsed:  57.040674924850464


,Company Names (Leave Blank if Using LinkedIn Company Mining)
0,Digital Health Intelligence Ltd
1,"Proteus Digital Health, Inc"
2,Digital Health & Care Innovation Centre
3,Digital Health Forward
4,Digital Health Coalition
5,Australian Digital Health Agency
6,Digital Health Institute for Transformation (D...
7,Digital Health Canada
8,Digital Health Solutions
9,NHS Digital
